## Yolo11 tests

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os , sys
sys.path.append('..')
from pathlib import Path
cur_path = Path(os.getcwd()).parent
sam2_path = cur_path / 'sam2_octron'
sys.path.append(cur_path.as_posix())
from matplotlib import pyplot as plt
import cmasher as cmr
import numpy as np
import seaborn as sns
sns.set_theme(style='white')
%config InlineBackend.figure_format = 'retina'

In [3]:
from octron.yolo_octron.yolo_octron import YOLO_octron
from octron.yolo_octron.helpers.yolo_checks import check_yolo_models
from octron.yolo_octron.helpers.training import collect_labels

In [4]:
base_path = Path('/Users/horst/Documents/python/OCTRON/octron/notebooks').parent
models_yaml_path = base_path / 'yolo_octron/yolo_models.yaml'

project_path = Path('/Users/horst/Downloads/octron_project_2')
assert project_path.exists()

### Organize training data 

In [5]:
# Run collect_labels in isolation. This would be triggered if a new project path is chosen
#label_dict = collect_labels(project_path)   
#label_dict

In [6]:
yolo_octron = YOLO_octron(models_yaml_path=models_yaml_path,
                          )

Models folder /Users/horst/Documents/python/OCTRON/octron/yolo_octron/models exists.
Model file /Users/horst/Documents/python/OCTRON/octron/yolo_octron/models/yolo11m-seg.pt exists. Skipping download.
Model file /Users/horst/Documents/python/OCTRON/octron/yolo_octron/models/yolo11l-seg.pt exists. Skipping download.
Model file /Users/horst/Documents/python/OCTRON/octron/yolo_octron/models/yolo11x-seg.pt exists. Skipping download.


In [ ]:
yolo_octron.project_path = project_path

In [ ]:
yolo_octron.models_dict 

In [ ]:
yolo_octron.prepare_labels(
                    prune_empty_labels=True, 
                    min_num_frames=10, 
                    verbose=False, 
)
prepare_polygons = yolo_octron.prepare_polygons()
# Process all yielded values
for no_entry, total_label_dict, label, frame_no, total_frames in prepare_polygons:
    # add progress reporting here 
    pass
    
print("Polygon preparation complete!")
yolo_octron.prepare_split(
                    training_fraction=0.7,
                    validation_fraction=0.15,
                    verbose=False,
)

In [ ]:
create_training_data = yolo_octron.create_training_data()
# Process all yielded values
for no_entry, total_label_dict, label, split, frame_no, total_frames in create_training_data:
    # add progress reporting here 
    pass
    
yolo_octron.write_yolo_config()

## YOLO training

In [ ]:
_ = yolo_octron.load_model('YOLO11l')

In [ ]:
yolo_octron.launch_tensorboard()

In [ ]:
train_results = yolo_octron.train(epochs=1)

In [ ]:
# # Load a model
# import time

# path_to_model = Path('/Users/horst/Documents/python/OCTRON/octron/yolo_octron/models/yolo11l-seg.pt')
# config_path = Path('/Users/horst/Downloads/octron_project_2/model/training_data/yolo_config.yaml')
# path_to_training = Path('/Users/horst/Downloads/octron_project_2/model/training_data')

# def on_train_start(trainer):
#     print("🥳  Training is starting!")

# def on_fit_epoch_end(trainer):
#     current_epoch = trainer.epoch + 1 
#     time_epoch = trainer.epoch_time
#     print(f"Time for epoch: {time_epoch}")
#     remaining_time = time_epoch * (no_epochs - current_epoch)   
#     finish_time = time.time() + remaining_time
#     finish_time_str = ' '.join(time.ctime(finish_time).split()[:-1])
#     print(f"Estimated time remaining: {remaining_time} seconds")    
#     print(f'Estimated finish time: {finish_time_str}')  
    
# from ultralytics import YOLO
# model = YOLO(path_to_model)  # load a pretrained model (recommended for training)
# model.add_callback("on_train_start", on_train_start)
# model.add_callback("on_fit_epoch_end", on_fit_epoch_end)
# no_epochs = 1

# # Train the model
# # https://docs.ultralytics.com/usage/cfg/#solutions-settings
# results = model.train(data=config_path, 
#                       save_dir=path_to_training.as_posix(),
#                       name='training',
#                       mode='segment',
#                       device='cpu',
#                       mask_ratio=4,
#                       epochs=no_epochs,
#                       imgsz=640,
#                       resume=False,
#                       plots=True,
#                       batch=.9,
#                       cache=False,
#                       save=True,
#                       save_period=15,
#                       project=None,
#                       exist_ok=True,
#                       # augmentation
#                       augment=True,
#                       hsv_v=.25,
#                       degrees=180,
#                       scale=.5,
#                       shear=2,
#                       flipud=.1,
#                       fliplr=.1,
#                       mosaic=1.0,
#                       copy_paste=.5,
#                       copy_paste_mode='mixup', 
#                       erasing=.25,
#                       crop_fraction=1.0,
#                       )

In [ ]:
# model = YOLO('/Users/horst/Downloads/octron_project/octron_training/yolo runs/segment/train/weights/last.pt')  #

In [ ]:
# metrics = model.val(device='cpu', plots=True)